In [22]:
import json
import sys
import numpy as np
import torch
import torch.utils.data as Data
from torch.utils.data import SubsetRandomSampler, DataLoader
import torch.nn as nn
import torch.optim as optim
from Model import Transformer
import time
import os
from PDB2Angles import extract_backbone_model
import sys
import os
from Bio.PDB import *
from Bio.PDB import PDBParser
from Bio.PDB import Superimposer
from Bio.PDB.DSSP import DSSP
from Bio.PDB.Atom import *
from Bio.PDB.Residue import *
from Bio.PDB.Chain import *
from Bio.PDB.Model import *
from Bio.PDB.Structure import *
# from Bio.PDB.Vector import *
from Bio.PDB.Entity import*
import math
from PeptideBuilder import Geometry
import PeptideBuilder
import numpy
from os import path
import warnings

In [ ]:
train_path = "data/source/train/"
test_path1= "data/source/test/CASP14/"
test_path2 = "data/source/test/CASP15/"
os.environ["LIBCIFPP_DATA_DIR"] = "/kuhpc/work/slusky/s300y051/dssp/libdssp/mmcif_pdbx"
#input
pre_relax = "/work/slusky/s300y051/relax/test/pre_relax"
relax = "/work/slusky/s300y051/relax/test/relax"
angles = ["psi", "phi", "omega", "CCN", "CNC", "NCC"]
minlength,maxlength = 5,37
# choose angle
for file in os.listdir(relax):
    #get file id
    file_id = file.split(".")[0].split("_relaxed")[0]
    angles_dir = "/home/s300y051/scratch/angles_refine_afdb/"+ file_id
    #check if file_id +".pdb" exists in pre_relax
    if file_id + ".pdb" not in os.listdir(pre_relax):# or os.path.exists(angles_dir):
        print(file_id + ".pdb not in pre_relax or angles already present")
        continue
    #get the angles of the file 
    #  os.mkdir(angles_dir)
    #both the relaxedd and pre-relaxed files will have an output directory with name by file_id
    model_structure_geo_pre_relax = extract_backbone_model(pre_relax + "/" + file_id + ".pdb", angles_dir + '/source.csv')
    model_structure_geo_relax = extract_backbone_model(relax + "/" + file, angles_dir + '/target.csv')

In [2]:

from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
from Bio.PDB.PDBIO import PDBIO
#import parser
from Bio.PDB import PDBParser

In [5]:
def get_secondary_structure(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_file)
    model = structure[0]
    dssp = DSSP(model, pdb_file, dssp='/kuhpc/work/slusky/s300y051/dssp/bin/mkdssp')
    ss_key = ''
    #replace any '-' with 'C'
    ss_key = [str(dssp[list(dssp.keys())[i]][2]) for i in range(len(dssp))]
    ss_key = [x if x != '-' else 'C' for x in ss_key]
    #replace G with H, I with H, B with E, T with C, S with C   
    for i, struc in enumerate(ss_key):
        if struc == 'G':
            ss_key[i] = 'H'
        elif struc == 'I':
            ss_key[i] = 'H'
        elif struc == 'B':
            ss_key[i] = 'E'
        elif struc == 'T':
            ss_key[i] = 'C'
        elif struc == 'S':
            ss_key[i] = 'C'
    return ss_key

In [12]:
parser = PDBParser()
structure = parser.get_structure('structure', "/home/s300y051/scratch/alphafoldSwiss/A0A3B6UEU3.pdb")
model = structure

In [25]:
#MODEL  to pdb
io = PDBIO()
io.set_structure(model)
io.save("/kuhpc/work/slusky/s300y051/AnglesRefine/example/try.pdb")
#delete pdb
import os
os.remove("/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/")

IsADirectoryError: [Errno 21] Is a directory: '/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/'

In [26]:
pdb_path = "/home/s300y051/scratch/alphafoldSwiss/A0A3B6UEU3.pdb"
structure=parser.get_structure('sample', pdb_path)
io = PDBIO()
io.set_structure(structure)

In [28]:
io.save("/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/" + pdb_path.split("/")[-1])

In [29]:
ss_key = get_secondary_structure("/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/" + pdb_path.split("/")[-1])

In [30]:
os.remove("/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/" + pdb_path.split("/")[-1])

In [31]:
from Bio.PDB.Entity import*
import math
from PeptideBuilder import Geometry
import PeptideBuilder
import numpy
from os import path
import warnings

In [32]:

resdict = { 'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', \
	    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', \
	    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', \
	    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y' }

In [33]:
model=structure[0]
model_structure_geo=[]
prev="0"
N_prev="0"
CA_prev="0"
CO_prev="0"
##O_prev="0"
prev_res=""
rad=180.0/math.pi
fh = open("/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/test1.csv", "w")
### now first print the headers,please filter them when you really load the file ### 
headers = "#,SS,residue,CA_C_N_angle,C_N_CA_angle,CA_N_length,CA_C_length,peptide_bond,psi_im1,omega phi,CA_N_length,CA_C_length,N_CA_C_angle\n"
fh.write(headers) 
for i, res in enumerate(model.get_residues()):
    if(res.get_resname() in resdict.keys()):
        geo=Geometry.geometry(resdict[res.get_resname()])
        if(prev=="0"):
                N_prev=res['N']
                CA_prev=res['CA']
                C_prev=res['C']
                ##O_prev=res['O']
                prev="1"
        else:
                n1=N_prev.get_vector()
                ca1=CA_prev.get_vector()
                c1=C_prev.get_vector()
                ##o1=O_prev.get_vector()
                        
                ##O_curr=res['O']
                C_curr=res['C']
                N_curr=res['N']
                CA_curr=res['CA']
                                            
                ##o=O_curr.get_vector()
                c=C_curr.get_vector()
                n=N_curr.get_vector()
                ca=CA_curr.get_vector()

                geo.CA_C_N_angle=calc_angle(ca1, c1, n)*rad
                geo.C_N_CA_angle=calc_angle(c1, n, ca)*rad
                geo.CA_N_length= CA_curr-N_curr
                geo.CA_C_length= CA_curr-C_curr
                geo.peptide_bond= N_curr-C_prev

                psi= calc_dihedral(n1, ca1, c1, n) ##goes to current res
                omega= calc_dihedral(ca1, c1, n, ca) ##goes to current res
                phi= calc_dihedral(c1, n, ca, c) ##goes to current res

                geo.psi_im1=psi*rad
                geo.omega=omega*rad
                geo.phi=phi*rad

                geo.CA_N_length= CA_curr - N_curr
                geo.CA_C_length= CA_curr - C_curr 
                ##geo.C_O_length= C_curr - O_curr

                geo.N_CA_C_angle= calc_angle(n, ca, c)*rad
                ##geo.CA_C_O_angle= calc_angle(ca, c, o)*rad

                ##geo.N_CA_C_O= calc_dihedral(n, ca, c, o)*rad

                N_prev=res['N']
                CA_prev=res['CA']
                C_prev=res['C']
                ##O_prev=res['O']
        # now write the angles to file 
        fh.write(str(i) + ",")
        fh.write(str(resdict[res.get_resname()]) + ",")
        fh.write(str(ss_key[i]) + ",")
        fh.write(str(geo.CA_C_N_angle) + ",")
        fh.write(str(geo.C_N_CA_angle) + ",") 
        fh.write(str(geo.CA_N_length) + ",")
        fh.write(str(geo.CA_C_length) + ",")
        fh.write(str(geo.peptide_bond)+ ",")
        fh.write(str(geo.psi_im1) + ",")
        fh.write(str(geo.omega) + ",")
        fh.write(str(geo.phi) + ",")
        fh.write(str(geo.CA_N_length) + ",")
        fh.write(str(geo.CA_C_length) + ",")
        fh.write(str(geo.N_CA_C_angle) + "\n")                   
        model_structure_geo.append(geo)
fh.close()
model_structure_geo

[CA_N_length = 1.46
 CA_C_length = 1.52
 N_CA_C_angle = 110.9416
 C_O_length = 1.23
 CA_C_O_angle = 120.4816
 N_CA_C_O_diangle = 120.0
 phi = -120
 psi_im1 = 140
 omega = 180.0
 peptide_bond = 1.33
 CA_C_N_angle = 116.642992978143
 C_N_CA_angle = 121.382215820277
 CA_CB_length = 1.52
 C_CA_CB_angle = 109.5
 N_C_CA_CB_diangle = 122.6733
 CB_CG_length = 1.52
 CA_CB_CG_angle = 113.68
 N_CA_CB_CG_diangle = -64.4
 CG_SD_length = 1.81
 CB_CG_SD_angle = 112.69
 CA_CB_CG_SD_diangle = -179.6
 SD_CE_length = 1.79
 CG_SD_CE_angle = 100.61
 CB_CG_SD_CE_diangle = 70.1
 residue_name = M,
 CA_N_length = 1.4707664
 CA_C_length = 1.5264422
 N_CA_C_angle = 108.40018231716597
 C_O_length = 1.23
 CA_C_O_angle = 120.54
 N_CA_C_O_diangle = 120.0
 phi = -81.60659669674149
 psi_im1 = 126.78817428669277
 omega = -179.5958447010252
 peptide_bond = 1.3380448
 CA_C_N_angle = 118.75886374072154
 C_N_CA_angle = 122.92669726152386
 CA_CB_length = 1.52
 C_CA_CB_angle = 109.5
 N_C_CA_CB_diangle = 122.76
 CB_CG_length 

In [10]:
os.environ["LIBCIFPP_DATA_DIR"] = "/kuhpc/work/slusky/s300y051/dssp/libdssp/mmcif_pdbx"
ss_key = extract_backbone_model("/home/s300y051/scratch/alphafoldSwiss/A0A068Q5Q5.pdb", "/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/bullshit.pdb")

KeyError: 0

In [7]:
ss_key = get_secondary_structure("/kuhpc/work/slusky/s300y051/AnglesRefine/tmp/B6HI95.pdb")

In [ ]:
parser = PDBParser()
structure = parser.get_structure('structure', "/work/slusky/s300y051/relax/test/relax/A0A023I7E1_relaxed_0001.pdb")[0]
#get num residues
num_residues = len(list(structure.get_residues()))

In [19]:
num_residues

796